In [1]:
import json

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    LocationMessageContent
)
import cwa


with open('env.json', encoding='utf-8') as f:
    env = json.load(f)

app = Flask(__name__)

cwa_key = env['key']
configuration = Configuration(access_token=env['CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['CHANNEL_SECRET'])


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    ask = event.message.text
    ans = cwa.tostr(cwa.cwa(ask, cwa_key), '\n')
    
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=ans)]
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_message(event):
    x, y = event.message.latitude, event.message.longitude
    ans = cwa.tostr(cwa.cwa((x, y), cwa_key), '\n')
    
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=ans)]
            )
        )

if __name__ == "__main__":
    app.run()

my __name__ is cwa
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Dec/2025 09:19:34] "POST /callback HTTP/1.1" 200 -


In [ ]:
import flask
import cwa

with open('env.json', encoding='utf-8') as f:
    env = json.load(f)

app = Flask(__name__)

cwa_key = env['key']

@app.route("/cwa", methods=['GET', 'POST'])
def cwa_():
    '''http://127.0.0.1:5000/cwa?site=臺北''' 
    site = flask.request.args.get('site')
    if site:
        return cwa.tostr(cwa.cwa(site, cwa_key), ', ')
    return 'Bad Request', 400

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


_load_sites() done


127.0.0.1 - - [02/Dec/2025 09:33:15] "GET /cwa?site=臺北 HTTP/1.1" 200 -
